In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
pd.set_option('display.max_colwidth', 1200)

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

## Study data train

In [6]:
df_pred = pd.read_csv("data/df_train_10_pred.csv")

In [7]:
df_pred.head()

,Unnamed: 0,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
0,0,dl,"del líquido una glucorraquia normal , proteinorraquia de 102 mg/ < start > dl < end > 960 células/mm3 un 90 % de las cuales se correspondían",decilitro,1,"[CLS] decilitro [SEP] del líquido una glucorraquia normal , proteinorraquia de 102 mg/ < start > dl < end > 960 células/mm3 un 90 % de las cuales se correspondían [SEP]",1.0,0.0,0.0
1,1,dl,"que eran de 5,8 g/dl con albúmina de 1.400 mg/ < start > dl < end > El elemental de orina y sedimento eran normales , así",decilitro,1,"[CLS] decilitro [SEP] que eran de 5,8 g/dl con albúmina de 1.400 mg/ < start > dl < end > El elemental de orina y sedimento eran normales , así [SEP]",1.0,0.0,0.0
2,2,dl,"67 y las proteínas totales que eran de 5,8 g/ < start > dl < end > con albúmina de 1.400 mg/dl . El elemental de orina",decilitro,1,"[CLS] decilitro [SEP] 67 y las proteínas totales que eran de 5,8 g/ < start > dl < end > con albúmina de 1.400 mg/dl . El elemental de orina [SEP]",1.0,0.0,0.0
3,3,dl,"las pruebas realizadas destacaba una anemia normocítica ( 9,9 g/ < start > dl < end > de Hb ) , leucocitos 6.700 con 84 % de",decilitro,1,"[CLS] decilitro [SEP] las pruebas realizadas destacaba una anemia normocítica ( 9,9 g/ < start > dl < end > de Hb ) , leucocitos 6.700 con 84 % de [SEP]",0.0,0.0,0.0
4,4,g,"gamma-GT 67 y las proteínas totales que eran de 5,8 < start > g < end > dl con albúmina de 1.400 mg/dl . El elemental de",gramo,1,"[CLS] gramo [SEP] gamma-GT 67 y las proteínas totales que eran de 5,8 < start > g < end > dl con albúmina de 1.400 mg/dl . El elemental de [SEP]",1.0,0.0,0.0


In [8]:
df_pred.shape

(6360, 9)

Veamos cuantos acrónimos son ambiguos

In [23]:
amb = df_pred.groupby('short_form')['long_form'].nunique().reset_index().sort_values('long_form', ascending = False)
print(amb.shape)
amb[amb['long_form'] == 1].shape

(768, 2)


(690, 2)

Filtramos para aquellas no ha acertado

In [9]:
wrong = df_pred[((df_pred['label'] == 0) & (df_pred['Prediction_train1'] == 1.0)) | (df_pred['label'] == 1) & (df_pred['Prediction_train1'] == 0.0)]

In [11]:
wrong.shape

(313, 8)

In [10]:
del wrong['Unnamed: 0']

In [12]:
wrong.sort_values(['short_form'])

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
365,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
370,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanina aminotransferaso,0,[CLS] alanina aminotransferaso [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0
366,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transferar,0,[CLS] alanine transferar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],1.0,0.0,0.0
369,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transferar,0,[CLS] alanine transferar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0
368,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transaminar,1,[CLS] alanine transaminar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
564,rpm,", taquicárdico ( 104 lpm ) y taquipneico ( 22 < start > rpm < end > . En fosa iliaca derecha se le palpaba un ``",respiracion minuto,1,"[CLS] respiracion minuto [SEP] , taquicárdico ( 104 lpm ) y taquipneico ( 22 < start > rpm < end > . En fosa iliaca derecha se le palpaba un `` [SEP]",0.0,0.0,0.0
566,s,"zonas sarcomatosas y negativo en zonas epiteliales puras , Proteína < start > s < end > 100 positivo en zonas sarcomatoides y negativo en las epiteliales",soluble,1,"[CLS] soluble [SEP] zonas sarcomatosas y negativo en zonas epiteliales puras , Proteína < start > s < end > 100 positivo en zonas sarcomatoides y negativo en las epiteliales [SEP]",0.0,0.0,0.0
567,s,"zonas sarcomatosas y negativo en zonas epiteliales puras , Proteína < start > s < end > 100 positivo en zonas sarcomatoides y negativo en las epiteliales",segundo,0,"[CLS] segundo [SEP] zonas sarcomatosas y negativo en zonas epiteliales puras , Proteína < start > s < end > 100 positivo en zonas sarcomatoides y negativo en las epiteliales [SEP]",1.0,0.0,0.0
604,β-HCG,"una elevación de la α-FTP ( 48,4 ng/mL ) , < start > β-HCG < end > normal ( < 0,6 mUI/mL ) y LDH normal .",subunidad beta human chorionic gonadotropin,1,"[CLS] subunidad beta human chorionic gonadotropin [SEP] una elevación de la α-FTP ( 48,4 ng/mL ) , < start > β-HCG < end > normal ( < 0,6 mUI/mL ) y LDH normal . [SEP]",0.0,0.0,0.0


In [14]:
wrong.groupby('short_form').size().reset_index()

,short_form,0
0,ALT,5
1,AOC,2
2,AST,5
3,AV,1
4,BA,1
...,...,...
73,pg,1
74,rpm,1
75,s,2
76,β-HCG,1


In [15]:
wrong[wrong['short_form'] == 'ALT']

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
365,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
366,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transferar,0,[CLS] alanine transferar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],1.0,0.0,0.0
368,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transaminar,1,[CLS] alanine transaminar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],0.0,0.0,0.0
369,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transferar,0,[CLS] alanine transferar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0
370,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanina aminotransferaso,0,[CLS] alanina aminotransferaso [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0


## Study data test

In [32]:
test = pd.read_csv("data/test_data_beto_10.csv")

In [34]:
test.iloc[2850]['context']

'la mutación 1691 G-A del factor V y 20210 G- < start > A < end > del factor II . El resto del estudio incluyó hemograma'

In [30]:
acr = 'PCR'

text = 'Nuria y Poveda se han hecho una PCR por la fiesta del sábado'

In [31]:
start_i = text.find(acr)
end_i = start_i + len(acr)

In [32]:
text[start_i:end_i]

'PCR'